In [1]:
'''
frame quality assessment(fqa) on the dataset 
'''
import os
import sys

from tqdm import tqdm
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data

import VideoTransforms as transforms
import arch.backbones as models
from utils.ReadData import Frame_dataset2 as dataset
from utils.FrameOperation import frame_processing
from arch.TemporalAggregation import gru
from prefetch_generator import BackgroundGenerator

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

class DataLoaderX(torch.utils.data.DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())    

    
def main(opts):
    global args, fp
    args = opts
    set_random_seed(args.seed)
    fp = frame_processing(num_station_points = args.num_station_points, 
                          process_type = args.process_type, alpha = args.alpha, modality = args.modality)

    val_dataset = dataset(root = args.data, phase = "val", modality = args.modality)
    val_loader = DataLoaderX(val_dataset,
                             batch_size = args.batch_size,
                             pin_memory = True,
                             shuffle = False,
                             num_workers = args.num_workers,)

    cnn = models.__dict__[args.arch](pretrained=False, num_classes=args.num_classes).cuda()
    par = torch.load(f"./ModelCheckpoints/backbones/{args.arch}_best.pth.tar", map_location = "cpu")["state_dict"]
    cnn.load_state_dict(par, strict=False)
    cnn.eval()
    print("CNN %s is loaded. " % (args.arch))
    
    net = gru(in_dim = args.in_dim, hidden_dim = args.hidden_dim, num_layers = args.num_layers, dp_ratio = args.dropout, 
                        action_classes = args.action_classes, num_classes = args.num_classes, fqa = args.fqa).cuda() 
    
    if args.pre_dic != "":
        sta = torch.load(args.pre_dic)
        weights_dict = sta['par']
        for k in list(weights_dict.keys()):
            if 'ops' in k or "params" in k:
                del(weights_dict[k])
        print(sta["epoch"])
        load_weights_dict = {k: v for k, v in weights_dict.items() if net.state_dict()[k].numel() == v.numel()}    
        print("lstm and policy network", net.load_state_dict(load_weights_dict))
    else:
        print("[WARMING] No pretrained parameter") 
        sys.exit()  
         
    net.eval()
    val_accuracy, fqa_dataset = validate_one_epoch(epoch = 0, val_loader = val_loader,
                                     net = net, cnn = cnn)
    
    torch.save(fqa_dataset, args.save_path)

def validate_one_epoch(epoch, val_loader, net, cnn):
    acc = 0.0
    val_num = len(val_loader)
    fqa_dataset = {}
    action_list = np.array([0]*args.action_classes)
    with torch.no_grad():
        val_bar = tqdm(val_loader)
        for _, val_data in enumerate(val_bar):
            actions, scores = [], []
            images, _, labels = val_data
            key = tuple(sum(images, []))
            step, action, feature, hidden, feature_station_point = 0, 0, 0, None, None
            action_fits = [0]*args.action_classes
            station_point, station_id = fp.get_station_point(frame_list = images)  
            feature_station_point = cnn(station_point.cuda(non_blocking = True), extract_feature = True)
            is_keep = True
            while is_keep:
                num = args.action_space[str(action)]
                changed_frame = fp.combine_frame(images[step:step+num], is_training = False, step = step)
                step += num
                next_station_id = next_station_point(station_id, step)
                feature = cnn(changed_frame.cuda(non_blocking = True), extract_feature = True)
                (action, score), hidden = net(feature, step, hidden = hidden, 
                               station_point = feature_station_point[next_station_id],
                               train_policy = True)
                actions.append(action)
                scores.append(score)
                is_keep =  step + args.action_space[str(action)] < len(images)    
                action_fits[action] += 1
            changed_frame = fp.combine_frame(images[step:], is_training = False, step = step)
            fqa_dataset[key] = (actions, scores)
            feature = cnn(changed_frame.cuda(non_blocking = True), extract_feature = True)
            val_p = net(feature, step, hidden = hidden, 
                              train_policy = True, check = True)
            val_predict = torch.max(val_p, dim= 1)[1]
            acc += torch.eq(val_predict, labels.cuda(non_blocking = True)).sum().item()
            val_bar.desc = "valid epoch[{}]".format(epoch + 1)
            action_list += np.array(action_fits)
            #print(action_fits)
    #raise
    print(action_list)
    print(action_list/sum(action_list))
    val_accuracy = acc/val_num
    print('[epoch %d] val_accuracy: %.4f' % (epoch + 1, val_accuracy))
    return val_accuracy, fqa_dataset

        
def next_station_point(station_point_id, step):
    for i in range(len(station_point_id)):
        if station_point_id[i] - step >= 0:
            return i   

def set_random_seed(the_seed):
    if the_seed >= 0:
        np.random.seed(the_seed)
        torch.manual_seed(the_seed)

In [ ]:
import argparse

if __name__ == '__main__':
    #是否预训练
    pretrain = True
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', type = int, default = 1007)
    parser.add_argument('--data', type = str, default = r"/home/bijianchao/FLAME/")
    parser.add_argument('--save_path', type = str, default = "./frame_selection/fqa.pth")
    parser.add_argument('--fqa', type = bool, default = True)
    parser.add_argument('--num_classes', type = int, default = 2)
    parser.add_argument('--action_classes', type = int, default = 4)
    parser.add_argument('--action_space', type = dict, default = {"0": 1, "1": 3, "2": 5, "3": 7})
    parser.add_argument('--batch_size', type = int, default = 1)
    parser.add_argument('--in_dim', type = int, default = 1280)
    parser.add_argument('--hidden_dim', type = int, default = 512)
    parser.add_argument('--num_layers', type = int, default = 1)
    parser.add_argument('--num_station_points', type = int, default = 2)
    parser.add_argument('--num_workers', type = int, default = 4)
    parser.add_argument('--alpha', type = float, default = 0.3, help = "hyperparameter of beta distribution used in mixup")
    parser.add_argument('--dropout', type = float, default = 0.5)
    parser.add_argument('--arch', '-a', metavar='ARCH', default='rgb_mobilenetv2',
                    choices=model_names,
                    help='model architecture: ' +
                        ' | '.join(model_names) +
                        ' (default: rgb_vgg16)')
    parser.add_argument('--modality', '-m', metavar='MODALITY', default='rgb',
                    choices=["rgb", "flow"],
                    help='modality: rgb | flow')
    parser.add_argument('--process_type', type = str, default = 'mixup', help = "methods to combine frames together")
    parser.add_argument('--pre_dic', type = str, default = r""  ,
                    help = 'weight of models')
    opt = parser.parse_args(args = [])
    
    if pretrain:
        main(opt)